<a href="https://colab.research.google.com/github/Vei-son/Colab-files/blob/main/%E2%80%9C%E5%8A%A8%E7%89%A9%E5%88%86%E7%B1%BB%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

动物分类教程

欢迎各位同学选修本课程！

使用 Python 进行编程文档请参阅 https://docs.python.org/zh-cn/3/contents.html 。

使用 PyTorch 框架，下载、文档等详情参阅 https://pytorch.org/ ，具体中文文档参阅 https://www.pytorch.wiki/ 。

## 环境配置

在运行本代码前，你需要在以下选项二选一完成：本地环境配置或使用 Google Colab * <a href="https://colab.research.google.com/drive/1Lf0Y5-3gvjKKggQTf1zWrEqHYYIVp3z2#scrollTo=Omm0IoVSJ4Kw" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
完成。若你使用 Colab 可以忽略以下配置环境的过程，可直接阅读下一节。

建议你下载 Anaconda 完成虚拟环境的配置，https://www.anaconda.com ，因为在不同的项目中需要的 Python 版本、不同包的版本可能不同，因此需要不同包的管理软件。Anaconda 的使用，参阅 https://anaconda.cloud/support-center ，或 https://zhuanlan.zhihu.com/p/348120084 获得对新手更为友好的中文教程。

Colab 使用建议：你可以使用你的 Google Drive，将数据集上传至 Google Drive 后，进行挂载使用。否则需要每次会话时重复上传。Google Drive 挂载在 Colab 使用参阅 https://blog.csdn.net/aiynmimi/article/details/88238246

# Baseline 算法

*更新（8.30）* 更新了直接下载数据集的代码。 

In [ ]:
!gdown --id 1_VGsRsHGGIdjg6gE3qq7YAJ0wcYTkfAz --output /content/Animal.zip
!chmod +x /content/Animal.zip
!unzip -o /content/Animal.zip

***! 表示让 colab 运行 Linux 命令，当然你也可以用以下方式完成，与上一个代码块的操作一致：***

In [ ]:
%%bash
gdown --id 1_VGsRsHGGIdjg6gE3qq7YAJ0wcYTkfAz --output /content/Animal.zip
chmod +x /content/Animal.zip
unzip -o /content/Animal.zip

## 数据读入

如何读入数据？参阅 https://pytorch.org/tutorials/beginner/basics/data_tutorial.html ，我们进行对数据集的读入。
在真正读入前，我们先进行一些预处理。

In [ ]:
NUM_CLASSES = 22
LABELS = [
    "ape", "bear", "bison", "cat", 
    "chicken", "cow", "deer", "dog",
    "dolphin", "duck", "eagle", "fish", 
    "horse", "lion", "lobster", "pig", 
    "rabbit", "shark", "snake", "spider", 
    "turkey", "wolf"
]
LABEL_MAP = {
    0: "ape", 1: "bear", 2: "bison", 3: "cat", 
    4: "chicken", 5: "cow", 6: "deer", 7: "dog",
    8: "dolphin", 9: "duck", 10: "eagle", 11: "fish", 
    12: "horse", 13: "lion", 14: "lobster", 
    15: "pig", 16: "rabbit", 17: "shark", 18: "snake", 
    19: "spider", 20:  "turkey", 21: "wolf"
}

你还需要把训练集进行分割，分割出 Valid 集

由于本任务数据的特殊性，可以直接使用 ImageFolder ，而不需要单独设计 Dataset 类，因此先定义 transforms, 参阅 https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html 。

In [ ]:
from torchvision import transforms

transform_labeled = None #!TODO: rewrite this statement
transform_val = None #!TODO: rewrite this statement

将训练、测试集读入 DataLoader

In [ ]:
import os

from torchvision.datasets import ImageFolder, VisionDataset
from torch.utils.data import DataLoader

data_path = '/content/Animals Dataset' #@param 
batch_size = 16 #@param
num_workers = 0 #@param

train_path = (os.path.join(data_path, 'train'))
test_path = (os.path.join(data_path, 'test'))

train_dataset = ImageFolder(
    train_path,
    transform_labeled)
test_dataset = None #!TODO: rewrite this statement

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers)
test_loader = None #!TODO: rewrite this statement

## 建立模型

本节介绍如何定义一个模型，参阅 https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html 

先进行预处理

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

定义模型类，注意到 PyTorch 是动态图，因此在数据进行前向传播时我们相当于已经定义了 Backpropagation 反向传播。

In [ ]:
class BersonNetwork(nn.Module):
  def __init__(self):
    super(BersonNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.c1 = nn.Conv2d(3, 20, 5, 2, 0)
    self.c2 = nn.Conv2d(20, 1, 5, 1, 0)
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(0, 128), #!TODO: Change 0 to a proper value
        #!Tips: You should calculate the number of neurons.
        nn.ReLU(),
        nn.Linear(128, 512),
        nn.ReLU(),
        nn.Linear(512, 0),#!TODO: Change 0 to a proper value
    )

  def forward(self, x):
    x = self.c1(x)
    x = F.relu(x) #!Question: What's the difference between torch.nn.relu() and torch.nn.F.relu()
    x = self.c2(x)
    x = F.relu(x)
    x = x.view(x.size(0), -1)
    logits = self.linear_relu_stack(x)
    return logits

这样模型就可以定义为：

In [ ]:
model = BersonNetwork().to(device)
print(model)
model.train()

设置模型训练超参数、损失函数与优化器

In [ ]:
learning_rate = 1e-3 #@param
batch_size = 64 #@param
epochs = 5 #@param
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

定义评价指标

In [ ]:
class AverageMeter(object):
  """Computes and stores the average and current value"""
  def __init__(self, name, fmt=':f'):
      self.name = name
      self.fmt = fmt
      self.reset()

  def reset(self):
      self.val = 0
      self.avg = 0
      self.sum = 0
      self.count = 0

  def update(self, val, n=1):
      self.val = val
      self.sum += val * n
      self.count += n
      self.avg = self.sum / self.count

  def __str__(self):
      fmtstr = '{name} {avg' + self.fmt + '}'
      return fmtstr.format(**self.__dict__)

def accuracy(output, target, topk=(1,)):
  """Computes the accuracy over the k top predictions for the specified values of k"""
  with torch.no_grad():
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
losses = AverageMeter('Loss', ':.4e')
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')

训练

In [ ]:
import time

start = time.time()
for i in range(epochs):
  for batch, (X, y) in enumerate(train_loader):
    data_time.update(time.time() - start)
    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc1, acc5 = accuracy(pred, y, topk=(1, 5))
    losses.update(loss.item(), X.size(0))
    top1.update(acc1[0], X.size(0))
    top5.update(acc5[0], X.size(0))
    
  batch_time.update(time.time() - start)
  start = time.time()

  print(f"Epoch:{i + 1}: {batch_time}, {losses}, {top1}, {top5}")

由于模型过于简单，因此训练准确率不高。你可以尝试训练其它的模型（甚至于预训练模型，但预训练模型如何进行训练也是一个值得讨论的问题）。
同时，这里并没有使用 Valid 集来进行评价，而是使用了 train 集，你可以尝试加入 Valid 集（如提前分割，或用其他方式等）



# 输出结果

输出结果非常简单，与训练类似，但模型不能更新参数

In [ ]:
model.eval()
labels = []
with torch.no_grad():
  for _, (images, _) in enumerate(test_loader):
    y = model(images)
    batch_labels = torch.argmax(y, dim=1)
    labels.append(batch_labels)
ans = torch.cat(labels, 0).cpu().numpy()
print(ans)
print([LABEL_MAP[i] for i in ans])

# 保存模型

参阅 https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html

In [ ]:
torch.save(model, 'model.path')

In [ ]:
model = torch.load('model.path')